## 모듈화

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

## 모듈화


In [3]:
from retrievers import init_retriever
from states import GraphState
from rag import create_rag_chain
from nodes import *

# retriever 초기화
retriever = init_retriever()
# rag_chain 초기화
rag_chain = create_rag_chain()

## 그래프 생성

In [4]:
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

# 그래프 상태 초기화
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("query_expand", QueryRewriteNode())  # 질문 재작성
workflow.add_node("query_rewrite", QueryRewriteNode())  # 질문 재작성
workflow.add_node("web_search", WebSearchNode())  # 웹 검색
workflow.add_node("retrieve", RetrieveNode(retriever))  # 문서 검색
workflow.add_node("grade_documents", FilteringDocumentsNode())  # 문서 평가
workflow.add_node(
    "general_answer", GeneralAnswerNode(ChatOpenAI(model="gpt-4o", temperature=0))
)  # 일반 답변 생성
workflow.add_node("rag_answer", RagAnswerNode(rag_chain))  # RAG 답변 생성

# 엣지 추가
workflow.add_conditional_edges(
    START,
    RouteQuestionNode(),
    {
        "query_expansion": "query_expand",  # 웹 검색으로 라우팅
        "general_answer": "general_answer",  # 벡터스토어로 라우팅
    },
)

workflow.add_edge("query_expand", "retrieve")
workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_web_search_node,
    {
        "web_search": "web_search",  # 웹 검색 필요
        "rag_answer": "rag_answer",  # RAG 답변 생성 가능
    },
)

workflow.add_edge("query_rewrite", "rag_answer")

workflow.add_conditional_edges(
    "rag_answer",
    AnswerGroundednessCheckNode(),
    {
        "relevant": END,
        "not relevant": "web_search",
        "not grounded": "query_rewrite",
    },
)

workflow.add_edge("web_search", "rag_answer")


# 그래프 컴파일
app = workflow.compile(checkpointer=MemorySaver())

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

In [6]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid


def ask(question):
    # config 설정(재귀 최대 횟수, thread_id)
    config = RunnableConfig(
        recursion_limit=20, configurable={"thread_id": random_uuid()}
    )

    # 질문 입력
    inputs = {
        "question": question,
    }

    # 스트리밍 형식으로 그래프 실행
    stream_graph(
        app,
        inputs,
        config,
    )

In [ ]:
ask("대한민국의 수도는?")

In [ ]:
ask("Self-RAG 에서 사용되는 관련성 평가 노드 예제를 찾아줘")